# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [17]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
data = pd.read_csv('/content/drive/MyDrive/dataset (1)/df_cluster.csv')

In [20]:
data.head()

,TransactionID,AccountID,TransactionAmount,TransactionDate,TransactionType,Location,DeviceID,IP Address,MerchantID,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,PreviousTransactionDate,Cluster
0,TX000001,AC00128,14.09,2023-04-11 16:29:14,Debit,San Diego,D000380,162.198.218.92,M015,ATM,70.0,Doctor,81.0,1.0,5112.21,2024-11-04 08:08:08,0
1,TX000002,AC00455,376.24,2023-06-27 16:44:19,Debit,Houston,D000051,13.149.61.4,M052,ATM,68.0,Doctor,141.0,1.0,13758.91,2024-11-04 08:09:35,2
2,TX000003,AC00019,126.29,2023-07-10 18:16:08,Debit,Mesa,D000235,215.97.143.157,M009,Online,19.0,Student,56.0,1.0,1122.35,2024-11-04 08:07:04,0
3,TX000004,AC00070,184.50,2023-05-05 16:32:11,Debit,Raleigh,D000187,200.13.225.150,M002,Online,26.0,Student,25.0,1.0,8569.06,2024-11-04 08:09:06,0
4,TX000005,AC00411,13.45,2023-10-16 17:51:24,Credit,Atlanta,D000308,65.164.3.100,M091,Online,26.0,Student,198.0,1.0,7429.40,2024-11-04 08:06:39,0


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TransactionID            2512 non-null   object 
 1   AccountID                2512 non-null   object 
 2   TransactionAmount        2512 non-null   float64
 3   TransactionDate          2512 non-null   object 
 4   TransactionType          2512 non-null   object 
 5   Location                 2512 non-null   object 
 6   DeviceID                 2512 non-null   object 
 7   IP Address               2512 non-null   object 
 8   MerchantID               2512 non-null   object 
 9   Channel                  2512 non-null   object 
 10  CustomerAge              2512 non-null   float64
 11  CustomerOccupation       2512 non-null   object 
 12  TransactionDuration      2512 non-null   float64
 13  LoginAttempts            2512 non-null   float64
 14  AccountBalance          

In [22]:
data.describe()

,TransactionAmount,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,Cluster
count,2512.000000,2512.000000,2512.000000,2512.000000,2512.000000,2512.000000
mean,297.593778,44.673965,119.643312,1.124602,5114.302966,0.822054
std,291.946243,17.792198,69.963757,0.602662,3900.942499,0.992095
min,0.260000,18.000000,10.000000,1.000000,101.250000,0.000000
25%,81.885000,27.000000,63.000000,1.000000,1504.370000,0.000000
50%,211.140000,45.000000,112.500000,1.000000,4735.510000,0.000000
75%,414.527500,59.000000,161.000000,1.000000,7678.820000,2.000000
max,1919.110000,80.000000,300.000000,5.000000,14977.990000,3.000000


In [23]:
data.isnull().sum()

,0
TransactionID,0
AccountID,0
TransactionAmount,0
TransactionDate,0
TransactionType,0
Location,0
DeviceID,0
IP Address,0
MerchantID,0
Channel,0


In [24]:
data.drop(columns=['TransactionID','AccountID','TransactionDate','Location','DeviceID','IP Address','MerchantID','PreviousTransactionDate'], inplace=True)
data.head()

,TransactionAmount,TransactionType,Channel,CustomerAge,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,Cluster
0,14.09,Debit,ATM,70.0,Doctor,81.0,1.0,5112.21,0
1,376.24,Debit,ATM,68.0,Doctor,141.0,1.0,13758.91,2
2,126.29,Debit,Online,19.0,Student,56.0,1.0,1122.35,0
3,184.50,Debit,Online,26.0,Student,25.0,1.0,8569.06,0
4,13.45,Credit,Online,26.0,Student,198.0,1.0,7429.40,0


In [25]:

categorical_columns = data.select_dtypes(include=['object']).columns
for column in categorical_columns:
    label_encoder = LabelEncoder()
    data[column] = label_encoder.fit_transform(data[column])

In [26]:

numerical_columns = ['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'LoginAttempts', 'AccountBalance']
scaler = MinMaxScaler()
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [27]:
X = data.drop(columns=['Cluster'])
y = data['Cluster']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [35]:

models = {
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Logistic Regression': LogisticRegression(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC()
}


In [42]:

# Normalisasi data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Melatih dan mengevaluasi setiap model
for model_name, classifier in models.items():
    try:
        # Melatih model dengan data latih
        if model_name == "Logistic Regression":
            classifier.set_params(max_iter=500, solver='saga')  # Meningkatkan iterasi & solver alternatif

        classifier.fit(X_train_scaled, y_train)  # Menggunakan data yang sudah dinormalisasi

        # Memprediksi hasil menggunakan data uji
        y_pred = classifier.predict(X_test_scaled)

        # Menghitung akurasi model
        model_accuracy = accuracy_score(y_test, y_pred)

        # Membuat laporan klasifikasi
        class_report = classification_report(y_test, y_pred)

        # Membuat matriks kebingungan (confusion matrix)
        confusion_mat = confusion_matrix(y_test, y_pred)

        # Menampilkan hasil evaluasi
        print(f"Model: {model_name}")
        print(f"Akurasi: {model_accuracy:.4f}")
        print("Laporan Klasifikasi:\n", class_report)
        print("confusion matrix:\n", confusion_mat)
        print("=" * 50)

    except Exception as e:
        print(f"Model {model_name} gagal dieksekusi. Error: {e}")

Model: Decision Tree
Akurasi: 0.9940
Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       268
           1       0.98      0.98      0.98        53
           2       0.99      1.00      0.99       166
           3       1.00      0.94      0.97        16

    accuracy                           0.99       503
   macro avg       0.99      0.98      0.99       503
weighted avg       0.99      0.99      0.99       503

confusion matrix:
 [[267   0   1   0]
 [  0  52   1   0]
 [  0   0 166   0]
 [  0   1   0  15]]
Model: Random Forest
Akurasi: 0.9960
Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       268
           1       1.00      0.98      0.99        53
           2       0.99      1.00      0.99       166
           3       1.00      1.00      1.00        16

    accuracy                           1.00       503
   macro avg       1.00  

Tulis narasi atau penjelasan algoritma yang Anda gunakan.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.


## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.

### Analisis
1. Random Forest (Akurasi: 99.60%)

  * Model terbaik dengan akurasi tertinggi.

  * Precision, recall, dan F1-score hampir sempurna.

  * Tidak ada kesalahan klasifikasi untuk kelas 3.

  * Best Parameters: max_depth=None, min_samples_leaf=4, min_samples_split=2, n_estimators=50.

2. Decision Tree (Akurasi: 99.40%)

  * Hampir sebanding dengan Random Forest, tetapi lebih rentan terhadap overfitting.

  * Best Parameters: criterion=gini, max_depth=None, min_samples_leaf=1, min_samples_split=5.

3. Logistic Regression (Akurasi: 97.81%)

  * Model linier dengan performa baik, tetapi lebih rendah dari Decision Tree dan Random Forest.

  * Best Parameters: C=100, solver=lbfgs.

4. Support Vector Machine (Akurasi: 96.82%)

  * Performa cukup baik, tetapi lebih rendah dibanding model pohon keputusan.
  * Best Parameters: C=10, gamma=scale, kernel=linear.

5. K-Nearest Neighbors (Akurasi: 86.88%)
    
  * Performa terendah, terutama karena kesalahan klasifikasi di kelas 2.

  * Best Parameters: metric=euclidean, n_neighbors=5, weights=distance.